In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import format_number
from pyspark.sql import functions as f

ModuleNotFoundError: No module named 'pyspark.conf'

In [2]:
sc = SparkContext('local')
spark = SparkSession.builder.appName("Notes").getOrCreate()

In [4]:
x = sc.parallelize(['a', 'b'])

In [5]:
x

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [21]:
ix = spark.read.json('test.json')

In [28]:
x = spark.read.option('multiline', 'true').json('test copy.json')

In [29]:
ix.count()

1

In [30]:
ix.show()

+--------+-------+
| mission|   name|
+--------+-------+
|learning|Pyspark|
+--------+-------+



In [34]:
x.select('nested').show()

+------+
|nested|
+------+
|{1, 2}|
+------+



## workin on Walmart Data

In [87]:
from pyspark.sql import functions as f

In [97]:
# walmart = spark.read.format('csv').option('header', 'true').option('inferSchema','true').load('walmart_stock_data.csv')

walmart = spark.read.csv('walmart_stock_data.csv', header=True, inferSchema=True)

In [98]:
walmart.withColumn('modified_date', f.to_date(f.col("Date"),"MM-dd-yyyy")).show(2)

+----------+------+------+------+------+---------+-------------+
|      Date|  Open|  High|   Low| Close|   Volume|modified_date|
+----------+------+------+------+------+---------+-------------+
|11/23/2021|144.25|145.98|143.25|145.81|9,972,448|         null|
|11/22/2021|142.66|145.36| 142.0|144.78|9,716,788|         null|
+----------+------+------+------+------+---------+-------------+
only showing top 2 rows



In [122]:
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
walmart.select('date', f.to_date(f.col('date'), 'yyyy/mm/dd').alias('mod') ).show(1)

+----------+----+
|      date| mod|
+----------+----+
|11/23/2021|null|
+----------+----+
only showing top 1 row



In [42]:
walmart.filter('open == 144.25').show()

+----------+------+------+------+------+---------+
|      Date|  Open|  High|   Low| Close|   Volume|
+----------+------+------+------+------+---------+
|11/23/2021|144.25|145.98|143.25|145.81|9,972,448|
+----------+------+------+------+------+---------+



In [43]:
walmart.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume']

In [44]:
walmart.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: string (nullable = true)



In [46]:
walmart.describe().show()

+-------+----------+------------------+------------------+------------------+------------------+----------+
|summary|      Date|              Open|              High|               Low|             Close|    Volume|
+-------+----------+------------------+------------------+------------------+------------------+----------+
|  count|        22|                22|                22|                22|                22|        22|
|   mean|      null|147.77454545454546|148.67954545454543|146.64181818181817|147.60590909090908|      null|
| stddev|      null| 2.869992081078725| 2.529469127952351| 3.022299087597338|2.8208098450299897|      null|
|    min|10/25/2021|            142.14|            143.29|            140.86|            141.94|11,038,740|
|    max|11/23/2021|             152.0|             152.0|            149.83|            151.28| 9,972,448|
+-------+----------+------------------+------------------+------------------+------------------+----------+



In [47]:
summary = walmart.describe()

In [48]:
summary.show()

+-------+----------+------------------+------------------+------------------+------------------+----------+
|summary|      Date|              Open|              High|               Low|             Close|    Volume|
+-------+----------+------------------+------------------+------------------+------------------+----------+
|  count|        22|                22|                22|                22|                22|        22|
|   mean|      null|147.77454545454546|148.67954545454543|146.64181818181817|147.60590909090908|      null|
| stddev|      null| 2.869992081078725| 2.529469127952351| 3.022299087597338|2.8208098450299897|      null|
|    min|10/25/2021|            142.14|            143.29|            140.86|            141.94|11,038,740|
|    max|11/23/2021|             152.0|             152.0|            149.83|            151.28| 9,972,448|
+-------+----------+------------------+------------------+------------------+------------------+----------+



In [55]:
summary.select('summary',
    format_number(summary['Open'].cast('float'),2).alias('open')
).show()

+-------+------+
|summary|  open|
+-------+------+
|  count| 22.00|
|   mean|147.77|
| stddev|  2.87|
|    min|142.14|
|    max|152.00|
+-------+------+



In [56]:
walmart.show(5)

+----------+------+------+------+------+----------+
|      Date|  Open|  High|   Low| Close|    Volume|
+----------+------+------+------+------+----------+
|11/23/2021|144.25|145.98|143.25|145.81| 9,972,448|
|11/22/2021|142.66|145.36| 142.0|144.78| 9,716,788|
|11/19/2021|143.62| 144.0|141.94|142.39| 9,380,728|
|11/18/2021|142.14|143.29|140.86|143.16| 8,135,824|
|11/17/2021|143.16|144.72|141.82|141.94|11,038,740|
+----------+------+------+------+------+----------+
only showing top 5 rows



In [99]:
type(walmart.orderBy(walmart.High.desc()).select('Date').head(1)[0][0])

str

In [73]:
walmart.select(f.min('Date').alias('Min'), f.max('Date').alias('Max')).show()

+----------+----------+
|       Min|       Max|
+----------+----------+
|10/25/2021|11/23/2021|
+----------+----------+



In [76]:
walmart = walmart.withColumn('Year', f.year(walmart.Date))

In [77]:
walmart.show(5)

+----------+------+------+------+------+----------+----+
|      Date|  Open|  High|   Low| Close|    Volume|Year|
+----------+------+------+------+------+----------+----+
|11/23/2021|144.25|145.98|143.25|145.81| 9,972,448|null|
|11/22/2021|142.66|145.36| 142.0|144.78| 9,716,788|null|
|11/19/2021|143.62| 144.0|141.94|142.39| 9,380,728|null|
|11/18/2021|142.14|143.29|140.86|143.16| 8,135,824|null|
|11/17/2021|143.16|144.72|141.82|141.94|11,038,740|null|
+----------+------+------+------+------+----------+----+
only showing top 5 rows



In [3]:
df=spark.createDataFrame([["02-03-2013"],["05-06-2023"]],["input"])

In [8]:

df_f1 = spark.read.csv('file1.csv', header=True, inferSchema=True)
df_f2 = spark.read.csv('file2.csv', header=True, inferSchema=True)
df_f1.show()
df_f2.show()


+-----+------+---+--------+
|empid|  name|age|location|
+-----+------+---+--------+
|    1|lokesh| 25|     ctr|
|    2| madhu| 23|     tpt|
+-----+------+---+--------+

+-----+------+---+--------+
|empid|  name|age|location|
+-----+------+---+--------+
|    1|lokesh| 24|     ctr|
|    2| madhu| 23|     tpt|
|    3| ressh| 23|     vlr|
+-----+------+---+--------+



In [20]:
cols = ['age', 'location']
df_f2.alias('a').join(
    df_f2.alias('b'), on=['empid', 'name'], how='outer'
).select('empid', 'name', *(f.coalesce('b.'+col, 'a.'+col).alias(col) for col in cols)) \
.show()

+-----+------+---+--------+
|empid|  name|age|location|
+-----+------+---+--------+
|    1|lokesh| 24|     ctr|
|    3| ressh| 23|     vlr|
|    2| madhu| 23|     tpt|
+-----+------+---+--------+

